# Join Adjacency Matrix, Gene Expression & Labels

In [1]:
import pandas
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
import h5py

## Get Gene Expression

In [2]:
# params
TIMES_STD_THRESHOLD = 10.
MAX_ZEROS_ALLOWED = .5

In [3]:
# load data
gene_expression = pandas.DataFrame.from_csv('gene_expression/normalized-counts-labels.csv',
                                            encoding='utf-8', sep=',')
ge_nonames = gene_expression.drop('Name', axis=1)

# kick out super highly expressed genes
threshold = ge_nonames.mean(axis=1).std()*TIMES_STD_THRESHOLD
#print ("Threshold Gene Expression: {}".format(threshold))
anomalies = gene_expression[ge_nonames.mean(axis=1) > threshold]
ge_anomalies_removed = ge_nonames.drop(anomalies.index)

# kick out genes with too many zeros
ge_zeros_removed = ge_anomalies_removed[ge_anomalies_removed.astype('bool').mean(axis=1)>=(1-MAX_ZEROS_ALLOWED)]

# scaling
scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(ge_zeros_removed)
ge_scaled = pandas.DataFrame(scaled_features,
                             index=ge_zeros_removed.index,
                             columns=ge_zeros_removed.columns)

# print some information
print ("Had gene expression for {} genes in the beginning.".format(ge_nonames.shape[0]))
print ("kicked out {} super highly expressed genes".format(ge_nonames.shape[0] - ge_anomalies_removed.shape[0]))
print ("Kicked out {} genes with more than {}% zeros".format(ge_anomalies_removed.shape[0]-ge_zeros_removed.shape[0],
                                                             MAX_ZEROS_ALLOWED*100.
                                                            ))
print ("==> Left with gene expression for {} genes".format(ge_scaled.shape[0]))

Had gene expression for 41424 genes in the beginning.
kicked out 9 super highly expressed genes
Kicked out 15021 genes with more than 50.0% zeros
==> Left with gene expression for 26394 genes


## Get PPI Network according to Gene Expression Data

In [99]:
gene_names = gene_expression.loc[ge_scaled.index].Name.values
gene_ids = ge_scaled.index
print (gene_names.shape, gene_ids.shape)

(26394,) (26394,)


In [6]:
# read interaction data
interactions = pandas.DataFrame.from_csv('ppi_network/ConsensusPathDB_human_PPI.csv',
                                         header=1,
                                         sep='\t',
                                         encoding='utf8'
                                        )
interactions_nona = interactions.dropna()

# construct Adjacency Matrix
N = len(gene_names)
print (N)
adjacency_matrix = np.zeros((N, N), np.uint8)
adj_df = pandas.DataFrame(adjacency_matrix, index=gene_names, columns=gene_names)

count = 0
for index, row in interactions_nona.iterrows():
    if row.interaction_confidence > 0.5:
        interaction_partners = row.interaction_participants.split(',')
        if len(interaction_partners) == 2:
            i1 = interaction_partners[0].split('_')[0].strip() # get rid of "_HUMAN" at end of prot name
            i2 = interaction_partners[1].split('_')[0].strip()
            if i1 in gene_names and i2 in gene_names:
                adj_df.ix[i1, i2] = 1
                adj_df.ix[i2, i1] = 1
            
    count += 1
    if count % 10000 == 0:
        print ("Processed {} out of {} rows".format(count, interactions_nona.shape[0]))

26394
Processed 10000 out of 272744 rows
Processed 20000 out of 272744 rows
Processed 30000 out of 272744 rows
Processed 40000 out of 272744 rows
Processed 50000 out of 272744 rows
Processed 60000 out of 272744 rows
Processed 70000 out of 272744 rows
Processed 80000 out of 272744 rows
Processed 90000 out of 272744 rows
Processed 100000 out of 272744 rows
Processed 110000 out of 272744 rows
Processed 120000 out of 272744 rows
Processed 130000 out of 272744 rows
Processed 140000 out of 272744 rows
Processed 150000 out of 272744 rows
Processed 160000 out of 272744 rows
Processed 170000 out of 272744 rows
Processed 180000 out of 272744 rows
Processed 190000 out of 272744 rows
Processed 200000 out of 272744 rows
Processed 210000 out of 272744 rows
Processed 220000 out of 272744 rows
Processed 230000 out of 272744 rows
Processed 240000 out of 272744 rows
Processed 250000 out of 272744 rows
Processed 260000 out of 272744 rows
Processed 270000 out of 272744 rows


In [7]:
adj_df.sum().sum()

87244

## Get Labels

## Generate Training, Testing & Validation Splits

## Write to hdf5 File on Disk

In [111]:
gene_names_np = np.array(gene_names, dtype=object)
gene_ids_np = np.array(gene_ids, dtype=object)
gene_names_both = np.stack([gene_names_np, gene_ids_np], axis=1)

string_dt = h5py.special_dtype(vlen=str)

f = h5py.File('ppi_networks.h5', 'w')
f.create_dataset('consensusPathDB_ppi', data=adj_df, shape=adj_df.shape)
f.create_dataset('gene_expression', data=ge_scaled, shape=ge_scaled.shape)
f.create_dataset('gene_names', data=gene_names_both, dtype=string_dt)

# doesn't exist yet :-(
#f.create_dataset('y_train', data=y_train, shape=y_train.shape)
#f.create_dataset('y_test', data=y_test, shape=y_test.shape)
#f.create_dataset('y_val', data=y_val, shape=y_val.shape)

#f.create_dataset('mask_train', data=mask_train, shape=mask_train.shape)
#f.create_dataset('mask_test', data=mask_test, shape=mask_test.shape)
#f.create_dataset('mask_val', data=mask_val, shape=mask_val.shape)
f.close()

In [109]:
pandas.DataFrame(gene_names_both)

,0,1
0,TSPAN6,ENSG00000000003
1,DPM1,ENSG00000000419
2,SCYL3,ENSG00000000457
3,C1orf112,ENSG00000000460
4,FGR,ENSG00000000938
5,CFH,ENSG00000000971
6,FUCA2,ENSG00000001036
7,GCLC,ENSG00000001084
8,NFYA,ENSG00000001167
9,STPG1,ENSG00000001460


In [110]:
gene_expression

,Name,Pam3T16,Pam3T8,Pam3T16.1,Pam3T8.1,Pam3T16.2,Pam3T8.2,ControlT8,ControlT16,ControlT8.1,...,gfpmT8.1,gfpmT16.1,gfpmT8.2,gfpmT16.2,gfppT8,gfppT16,gfppT8.1,gfppT16.1,gfppT8.2,gfppT16.2
Ensembl-ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,TSPAN6,6.736327,8.519666,10.773706,10.813725,2.427830,7.425264,9.417385,7.920964,10.812953,...,12.529836,6.769370,13.440488,5.248289,14.583375,13.538040,18.220493,14.908853,16.156797,30.745221
ENSG00000000419,DPM1,97.676739,73.025712,132.362676,110.840686,87.401897,87.865624,92.725018,105.132797,121.218890,...,127.983328,147.421826,104.163784,86.596770,298.959192,239.172041,268.101547,155.611158,283.898001,247.298513
ENSG00000000457,SCYL3,84.204085,74.242807,61.564035,68.486928,78.499852,92.815800,86.929705,72.728852,66.585024,...,76.074006,63.180782,117.604272,94.469203,83.125239,79.724014,76.786365,62.430824,85.400212,93.572410
ENSG00000000460,C1orf112,61.468982,48.683808,70.798641,63.080065,63.123592,76.727728,68.819350,68.408327,77.397977,...,56.384263,72.958761,81.482960,51.608176,24.791738,33.092987,29.933668,59.635414,30.005480,26.734974
ENSG00000000938,FGR,53.890614,150.919804,143.136382,362.259802,156.999704,160.880721,62.299622,61.207450,189.511222,...,333.830638,257.988195,186.486774,70.851902,129.792040,97.774734,272.005939,245.996082,140.794944,101.592903
ENSG00000000971,CFH,18.524899,21.907714,43.094825,53.167483,29.943242,23.513336,4.346485,2.880351,15.365775,...,11.634848,17.299500,6.720244,6.997719,5.833350,7.521133,5.205855,11.181640,0.000000,1.336749
ENSG00000001036,FUCA2,90.098371,59.637665,84.650549,82.905228,84.164790,89.103168,119.528344,123.134988,144.552104,...,107.398597,136.139543,103.323753,80.473766,141.458740,118.833907,96.308323,109.952794,114.251635,98.919405
ENSG00000001084,GCLC,317.449401,109.538568,169.301098,410.020423,384.406491,128.704577,262.237943,301.716724,393.250014,...,367.840193,368.554564,218.407934,289.530614,457.917983,544.530055,183.506399,322.403956,459.314652,493.260277
ENSG00000001167,NFYA,225.666948,178.912994,155.449190,196.449345,191.798607,198.007041,163.717611,126.015338,156.503262,...,139.618175,173.747152,177.246439,165.321106,354.376018,427.200375,428.181597,230.155425,371.606327,442.463826
